# ch09. 装饰器 和 闭包

> 如果想掌握装饰器，那么必须理解闭包，即捕获函数主体外部定义的变量，也需要掌握 `nonlocal`

- Python 如何求解装饰器句法
- Python 如何判断变量是否为局部
- 闭包存在的原因和工作原理
- nonlocal 能解决什么问题
- 实现行为良好的装饰器
- 标准库中强大的装饰器：`@cache`, `@lru_cache`, `@singledispatch`;
- 实现一个参数化装饰器

## 装饰器的基础知识

装饰器是一种可调用对象，其参数是另一个函数（被装饰的函数）
装饰器可能会对被装饰的函数进行处理，然后返回函数，或者把函数替换为另一个函数/可调用对象

假如有一个名为 decorate 的装饰器

```python
@decorate
def target():
    print('running target()')
```

那么效果等同于下面的写法

```python
def target():
    print('running target()')

target = decorate(target)
```

<mark>装饰器通常会把一个函数替换为另一个函数</mark>

In [1]:
def deco(func):
    def inner():
        print('running inner()')
    return inner

@deco
def target():
    print('running target()')

target()

running inner()


调用被装饰的 `target()`，运行的其实是 `inner`

In [2]:
target

<function __main__.deco.<locals>.inner()>

查看对象，target 现在是 inner 的引用

<mark>严格来说，装饰器只是语法糖。装饰器可以像常规的可调用对象那样调用，传入另一个函数</mark>

**<span style="color:green">装饰器有以下三个基本性质</span>**

- 装饰器是一个函数或其他可调用对象

- 装饰器可以把装饰的函数替换成别的函数

- 装饰器在加载模块时立即执行

## Python 何时执行装饰器

> 装饰器的一个关键的性质是：<mark>装饰器在被装饰的函数定义之后立即运行</mark>，通常是在导入时（比如加载模块时）

以下为一个示例 `registration.py`

In [3]:
registry = []       # 保存被 @register 装饰的函数的引用

def register(func):
    print(f'running register({func})')      # 为了演示，显示被装饰的函数
    registry.append(func)
    return func

@register
def f1():
    print('running f1()')

@register
def f2():
    print('running f2()')

def f3():
    print('running f3()')


def main():
    print('running main()')
    print('registry ->', registry)
    f1()
    f2()
    f3()

if __name__ == '__main__':
    main()


running register(<function f1 at 0x107ef2a70>)
running register(<function f2 at 0x107ef2b90>)
running main()
registry -> [<function f1 at 0x107ef2a70>, <function f2 at 0x107ef2b90>]
running f1()
running f2()
running f3()


- `main()` 函数中首先显示 registry, 然后调用 f1(), f2(), f3()
- 只有把上面的示例作为**脚本运行**时，才能调用 `main()`

- 注意，register 在模块中其他函数之前运行了两次
- 在调用 register 时，传给它的参数是被装饰的函数

如果作为模块导入上面的文件，并不作为脚本运行，则输出如下的内容

```python
>>> import registration
running register(<function f1 at 0x107ef2a70>)
running register(<function f2 at 0x107ef2b90>)

```

<span style="color: green">上面的示例想强调的是，函数装饰器在导入模块时立即执行，而被装饰的函数只在显示调用时运行 </span>

## 注册装饰器

- 装饰器通常在一个模块中定义，然后再应用到其他模块中的函数上
- 大多数装饰器会在内部定义一个函数，然后将其返回

<mark>大多数装饰器会更改被装饰的函数。 通常的做法是，返回在装饰器内部定义的函数，取代被装饰的函数。</mark>

涉及内部函数的代码基本上离不开闭包，而理解闭包，需要熟悉Python中的变量作用域规则

## 变量作用域规则

看一个示例，关于全局变量和局部变量的

In [1]:
b = 6
def foo(a):
    print(a)
    print(b)
    b = 9

In [2]:
foo(3)

3


UnboundLocalError: local variable 'b' referenced before assignment

如上所示，代码竟然报错了，报错信息为：`local variable 'b' referenced before assignment`
并不如预期的首先打印 b 的值：6，然后将 b 赋值 9，因为 b 是全局变量，而且局部变量在打印之后才赋值

<span style="color: green">Python 编译函数主体时，判断 b 是局部变量，因为在函数内给它赋值了。生成的字节码证实了这种判断</span>


所以，Python会尝试从局部作用域中获取 b。调用 foo(3) 时，foo 的主体顺利获取并打印了局部变量 a 的值，但是尝试获取局部变量 b 的值时，发现 b 没有绑定值

> 在函数中赋值时，如果想让解释器把 b 当作全局变量，为它分配一个新值，就要使用 global 声明

In [1]:
b = 6
def foo(a):
    global b
    print(a)
    print(b)
    b = 9

In [2]:
foo(3)

3
6


In [3]:
print(b)

9


变量可能出现在三种作用域中
- 全局作用域
- 局部作用域
-  “非局部”作用域，这个作用域时闭包的基础

In [4]:
from dis import dis
dis(foo)

  4           0 LOAD_GLOBAL              0 (print)
              2 LOAD_FAST                0 (a)
              4 CALL_FUNCTION            1
              6 POP_TOP

  5           8 LOAD_GLOBAL              0 (print)
             10 LOAD_GLOBAL              1 (b)
             12 CALL_FUNCTION            1
             14 POP_TOP

  6          16 LOAD_CONST               1 (9)
             18 STORE_GLOBAL             1 (b)
             20 LOAD_CONST               0 (None)
             22 RETURN_VALUE


上面的反汇编示例

首先加载全局名称 print

加载局部名称 a

加载全局名称 b

## 闭包

闭包是延伸了作用域的函数，包括函数主体 f 中引用的非全局变量和局部变量。

这些变量必须来自函数 f 的外部函数的局部作用域

如果有一个函数作用是去计算不断增加系列的平均值，例如，计算整个历史中某个商品的平均收盘价
新价格每天都在增加，因此计算平均值要考虑目前为止的所有价格

可以看看 avg 函数的用法

```python
>>>avg(10)
10.0
>>>avg(11)
10.5
>>>avg(12)
11.0

```

一种比较简单的方式使用基于类的实现

average_oo.py 一个计算累计平均值的类

In [1]:
class Averager():
    def __init__(self):
        self.series = []
        
    def __call__(self, new_value):
        self.series.append(new_value)
        total = sum(self.series)
        return total / len(self.series)


In [2]:
avg = Averager()

res1 = avg(10)
print(res1)

res2 = avg(11)
print(res2)

res3 = avg(12)
print(res3)

10.0
10.5
11.0


下面这个是函数式实现，使用了高阶函数 `make_averager`

averager.py 一个计算累计平均值的高阶函数

In [3]:
def make_averager():
    series = []
    
    def averager(new_value):
        series.append(new_value)
        total = sum(series)
        return total / len(series)
    
    return averager

调用 make_averager, 返回一个 averager 函数对象。每次调用，averager 都会把参数添加到系列值中，然后计算当前平均值，如下所示

In [4]:
avg = make_averager()

res1 = avg(10)
print(res1)

res2 = avg(11)
print(res2)

res3 = avg(12)
print(res3)

10.0
10.5
11.0


上面两个示例有相似之处，调用 Averager() 或 make_averager() 得到一个可调用对象 avg，他会更新历史值，然后计算当前平均值。

第一个示例中，avg 是 Averager 类的实例；

第二个示例中，avg 是内部函数 averager

在第二个示例中，avg 在哪里寻找 series 呢？

series 是 make_averager 函数的局部变量，因为赋值语句 series = [] 在 make_averager 函数的主体中，但是，调用 avg(10) 时，make_averager 函数已经返回，局部作用域早就没了

如下图所示，在 averager 函数中，series 是自由变量，自由变量是一个术语，指未在局部作用域中绑定的变量。

![averager](./assets/ch09/free-var.png)

averager 函数的闭包延伸到自身的作用域之外，包含自由变量 series 的绑定

查看返回的 averager 对象，在 __code__ 属性（编译后的函数主体）保存着局部变量和自由变量的名称

In [5]:
varnames = avg.__code__.co_varnames
print(varnames)

freevars = avg.__code__.co_freevars
print(freevars)

('new_value', 'total')
('series',)


series 的值在返回的 avg 函数的 `__closure__` 属性中。`avg.__closure__` 中的各项对应 `avg.__code__.co_freevars` 中的一个名称。这些项是 cell 对象，有一个名为 cell_contents 的属性，保存着真正的值

In [6]:
freevars = avg.__code__.co_freevars
print(freevars)

closure = avg.__closure__
print(closure)

rel_val = avg.__closure__[0].cell_contents
print(rel_val)

('series',)
(<cell at 0x1061b14e0: list object at 0x1061c8040>,)
[10, 11, 12]


<mark>闭包是一个函数，保留着定义函数时存在的自由变量的绑定</mark>

如此以来，调用函数时，虽然定义作用域不可用了，但是仍可以使用那些绑定

<mark>注意，只有嵌套在其他函数中的函数才可能需要处理不在全局作用域中的外部变量，这些外部变量位于外层函数的局部作用域内</mark>

## nonlocal 声明

前面实现的 make_averager 函数的方法效率不高，我们把所有值存储在历史数列中，然后在每次调用 averager 时使用 sum 求和，更好的实现方式是，只存储目前的总值和项数，根据这两个数计算平均值

下面是一个有缺陷的实现方式

In [7]:
def make_averager():
    count = 0
    total = 0
    
    def averager(new_value):
        count += 1
        total += new_value
        return total / count
    
    return averager

调用之后，运行效果如下

In [8]:
avg = make_averager()

res = avg(10); print(res)

UnboundLocalError: cannot access local variable 'count' where it is not associated with a value

报错信息提示：无法访问本地变量 count，因为他没有关联一个值

实际上，我们在 averager 的主体中为 count 赋值了，这会把 count 变为局部变量，total 变量也是相同的问题

而之前的类实现没有此问题是因为，没有给 series 赋值，只是调用了 series.append, 并把它传给了 sum 和 len。即利用了 **列表是可变对象**的特性

但是，数值，字符串，元组等不可变类型只能读取，不能更新。

如果像 count += 1, 这其实会尝试重新绑定，会隐式创建局部变量 count。这样，count 不是自由变量，也就不会保存在闭包中。

为了解决这个问题，Python3 引入了 nolocal 关键字，作用是把变量标记为**自由变量**，即使在函数中为变量赋予了新值，闭包中保存的绑定也会随之更新

下面的示例使用 nonlocal 进行修正

In [9]:
def make_averager():
    count = 0
    total = 0
    
    def averager(new_value):
        nonlocal count, total
        count += 1
        total += new_value
        return total / count
    
    return averager


In [10]:
avg = make_averager()

res1 = avg(10); print(res1)
res2 = avg(11); print(res2) 

10.0
10.5


可以看到正常运行，学会使用 nonlocal 之后，来总结一下 Python 查找变量的方式

<mark>变量查找逻辑</mark>

Python 字节码编译器根据以下规则获取函数主体中出现的变量 x
- 如果是 `global x` 声明，则 x 来自模块全局作用域，并赋予那个作用域中 x 的值
- 如果是 `nonlocal x` 声明，则 x 来自最近一个定义它的外层函数，并赋予那个函数局部变量 x 的值
- 如果 x 是参数，或者在函数主体中赋了值，那么 x 就是局部变量
- 如果引用了 x，但是没有赋值也不是参数，则遵循以下规则
  - 在外层函数主体的局部作用域内查找 x
  - 如果在外层作用域内未找到，则从模块全局作用域内读取
  - 如果在模块全局作用域内未找到，则从 `__builtins__.__dict__` 中读取

## 实现一个简单的装饰器

下面定义了一个装饰器，该装饰器会在每次调用被装饰的函数时计时，把运行时间，传入的参数和调用的结果打印出来

clockdeco0.py: 一个会显示函数运行时间的简单的装饰器

In [11]:
import time

def clock(func):
    def clocked(*args):     # 定义内部函数 clocked，它接受任意个位置参数
        t0 = time.perf_counter()
        result = func(*args)    # 这行代码有效，是因为 clocked 的闭包中包含自由变量 func
        elapsed = time.perf_counter() - t0
        name = func.__name__
        arg_str = ', '.join(repr(arg) for arg in args)
        print(f'[{elapsed:0.8f}] {name}({arg_str}) -> {result!r}')
        return result
    
    return clocked


使用 clock 装饰器

In [13]:
import time


@clock
def snooze(seconds):
    time.sleep(seconds)


@clock
def factorial(n):
    return 1 if n < 2 else n * factorial(n-1)

if __name__ == '__main__':
    print('*' * 40, 'Calling snooze(.123)')
    snooze(.123)
    print('*' * 40, 'Calling factorial(6)')
    print('6! =', factorial(6))

**************************************** Calling snooze(.123)
[0.12727725] snooze(0.123) -> None
**************************************** Calling factorial(6)
[0.00000050] factorial(1) -> 1
[0.00001133] factorial(2) -> 2
[0.00001908] factorial(3) -> 6
[0.00002758] factorial(4) -> 24
[0.00003525] factorial(5) -> 120
[0.00004371] factorial(6) -> 720
6! = 720


<mark>工作原理</mark>

如前所述，以下内容

```python
@clock
def factorial(n):
    return 1 if n < 2 else n*factorial(n-1)

```
其实等价于以下内容

```python
def factorial(n):
    return 1 if n < 2 else n * factorial(n-1)


factorial = clock(factorial)
```


也就是说，在这两种情况下，factorial 函数都作为 func 参数传递给 clock 函数，clock 函数返回 clocked 函数，然后 Python 解释器把 clocked 赋值给 factorial 

factorial 保存的其实是 clocked 函数的引用。

自此以后，每次调用 factorial(n) 执行的都是 `clocked(n)`

clocked 大致做了下面几件事
- 记录初始时间 t0
- 调用原来的 factorial 函数，保存结果
- 计算运行时间
- 格式化收集的数据，然后打印出来
- 返回第二步保存的结果

<span style="color:green">这是装饰器的典型行为：</span>把被装饰的函数替换为新函数，新函数接受的参数与被装饰的函数一样，而且（通常）会返回被装饰的函数本该返回的值，同时还会做一些额外操作

上面实现的装饰器存在几个缺点：
- 不支持关键字参数
- 遮盖了被装饰函数的 `__name__` `__doc__` 属性

<mark>下面示例使用 `functions.wraps`装饰器把相关的属性从 func 身上复制到了 clocked 中</mark>

clockdeco.py: 改进后的 clock 装饰器

In [15]:
import time
from functools import wraps

def clock(func):
    @wraps(func)
    def clocked(*args, **kwargs):
        t0 = time.perf_counter()
        result = func(*args, **kwargs)
        elapsed = time.perf_counter() - t0
        name = func.__name__
        arg_lst = [repr(arg) for arg in args]
        arg_lst.extend(f'{k}={v!r}' for k,v in kwargs.items())
        arg_str = ', '.join(arg_lst)
        print(f'[{elapsed:0.8f}] {name}({arg_str}) -> {result!r}')
        return result
    
    return clocked


`functools.wraps` 只是标准库中开箱即用的装饰器之一

## 标准库中的装饰器

Python 内置了 3 个用于装饰方法的函数: `property`、`classmethod` 和 `staticmethod`。

- `functools.wraps` 的作用是协助构建行为良好的装饰器
- 标准库中最吸引人的几个装饰器
  - `cache`
  - `lru_cache`
  - `singledispatch`

### 使用 functools.cache 做备忘

它是一项优化技术，能把耗时的函数得到的结果保存起来，避免传入相同的参数时重复计算

> `functools.cache`是 Python3.9 新增的，如果想使用低版本，需要把 `@cache` 换成 `@lru_cache`

In [16]:
from functools import cache

@cache  # 需要 Python 3.9 及以上
@clock  # 这里叠放了装饰器，@cache 会应用到 @clock 返回的函数上
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n-2) + fibonacci(n-1)

if __name__ == '__main__':
    print(fibonacci(6))

[0.00000042] fibonacci(0) -> 0
[0.00000033] fibonacci(1) -> 1
[0.00003871] fibonacci(2) -> 1
[0.00000046] fibonacci(3) -> 2
[0.00004858] fibonacci(4) -> 3
[0.00000042] fibonacci(5) -> 5
[0.00005758] fibonacci(6) -> 8
8


<mark>叠放装饰器</mark>

如果想理解叠放装饰器，需要记住一点:
<span style="color:green"> `@` 是一种语法糖，其作用是把装饰器函数应用到下面的函数上 </span>

多个装饰器的行为就像调用嵌套函数一样。

```python
@alpha
@beta
def my_fn():
    ...
```

等同于以下内容

```python
my_fn = alpha(beta(my_fn))
```

也就是说，首先应用 beta 装饰器，然后再把返回的函数传给 alpha

<mark>被装饰的函数所接受的参数必须克哈希，因为底层 lru_cache 使用 dict 存储结果，字典的键取自传入的位置参数和关键字参数</mark>

除了优化递归算法，@cache 在从远程 API 中获取信息的应用程序中也能发挥巨大作用

<span style="color:green"> 如果缓存较大，则 `functools.cache` 有可能耗尽所有可用内存 </span>

`@cache` 更适合短期运行的命令行脚本使用，对于长期运行的进程，推荐使用 `functools.lru_cache`，并合理设置 maxsize 参数

---

### 使用 lru_cache

`functools.cache` 装饰器只是对较旧的 `functools.lru_cache` 函数的简单包装。
`functools.cache` 更加灵活，而且兼容旧版本

@lru_cache 的主要优势是可以通过 maxsize 参数限制内存用量上限。maxsize 参数的默认值相当保守，只有 128，即缓存最多只能有 128 条。

从 3.8 开始，lru_cache 有两种使用方法

```python
@lru_cache
def costly_func(a, b):
    ...

```

另一种方式是从 3.2 开始支持的加上 () 作为函数调用

```python
@lru_cache()
def costly_func(a, b):
    ...

```

两种用法都采用以下默认参数：
- maxsize=128
- typed=False

第一个可以设定存储多少条目，缓存满了之后，最不常用的条目会被丢弃，为新条目腾出空间，为了得到最佳性能，应该将 maxsize 设置为 2 的次方，如果传入 maxsize=None，则 LRU 将被彻底禁用，因此缓存速度更快，条目永远不会被丢弃，可能消耗过多内存，如 `functools.cache`

typed 参数决定是否把不同参数类型得到的结果分开保存，默认情况下，不分开，比如调用 f(1) 和 f(1.0) 被认为是调用一个缓存条目，若设为 typed=True，则不同的条目中存储可能不一样的结果

```python
@lru_cache(maxsize=2**20, typed=True)
def costly_func(a, b):
    ...

```

### 单分派泛化函数